In [1]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 28.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [7]:
import tempfile
import logging
import random
import os
import yaml
import time
import torch
import transformers
import pandas as pd

In [8]:
from transformers import AutoTokenizer,AutoModelForCausalLM,TrainingArguments,Trainer
import pandas as pd
import datasets

In [9]:
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-70m')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [55]:
data1 = '/content/mentalillnesses.json'
df = pd.read_json(data1)
df1 = df.to_dict()
df1

{'question': {0: 'What is depression?',
  1: 'What are the symptoms of anxiety disorders?',
  2: 'What is schizophrenia?',
  3: 'What is bipolar disorder?',
  4: 'What is obsessive-compulsive disorder (OCD)?',
  5: 'What causes post-traumatic stress disorder (PTSD)?',
  6: 'What are eating disorders?',
  7: 'What is anorexia nervosa?',
  8: 'What is bulimia nervosa?',
  9: 'What is attention-deficit/hyperactivity disorder (ADHD)?',
  10: 'What is autism spectrum disorder (ASD)?',
  11: 'What are personality disorders?',
  12: 'What is borderline personality disorder (BPD)?',
  13: 'What is antisocial personality disorder (ASPD)?',
  14: 'What is histrionic personality disorder?',
  15: 'What is narcissistic personality disorder (NPD)?',
  16: 'What is dissociative identity disorder (DID)?',
  17: 'What is generalized anxiety disorder (GAD)?',
  18: 'What is social anxiety disorder (social phobia)?',
  19: 'What is panic disorder?',
  20: 'What is insomnia?',
  21: 'What is seasonal aff

In [56]:
for i in range(len(df1['question'])):
  if 'question' in df1 and 'answer' in df1:
    example = df1['question'][i] + df1['answer'][i]

In [57]:
prompt_template = """
###answer the following question with the user data
{question}
###Answer:
"""

In [58]:
data_q_a =[]
for i in range(len(df1['question'])):
  question = df1['question'][i]
  answer = df1['answer'][i]
  text = prompt_template.format(question = question)
  data_q_a.append({'question':text,'answer':answer})

In [59]:
data_q_a

[{'question': '\n###answer the following question with the user data\nWhat is depression?\n###Answer:\n',
  'answer': 'Depression is a common mental disorder characterized by persistent sadness and a lack of interest or pleasure in previously rewarding or enjoyable activities.'},
 {'question': '\n###answer the following question with the user data\nWhat are the symptoms of anxiety disorders?\n###Answer:\n',
  'answer': 'Symptoms of anxiety disorders include excessive worry, restlessness, fatigue, difficulty concentrating, irritability, muscle tension, and sleep disturbances.'},
 {'question': '\n###answer the following question with the user data\nWhat is schizophrenia?\n###Answer:\n',
  'answer': 'Schizophrenia is a severe mental disorder that affects how a person thinks, feels, and behaves, characterized by delusions, hallucinations, and disorganized speech.'},
 {'question': '\n###answer the following question with the user data\nWhat is bipolar disorder?\n###Answer:\n',
  'answer': '

In [16]:
def tokenizing_function(examples):

    if "question" in examples and "answer" in examples:
      text = examples["question"][0] + examples["answer"][0]

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs

In [60]:
finetuning_dataset= datasets.load_dataset("json", data_files=data1, split="train")

tokenized_dataset = finetuning_dataset.map(
    tokenizing_function,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [61]:
print(tokenized_dataset)

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 31
})


In [62]:
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

In [63]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 27
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4
    })
})

In [84]:
base_model = AutoModelForCausalLM.from_pretrained('EleutherAI/pythia-410m')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

In [85]:
import torch
device_count = torch.cuda.device_count()
if device_count > 0:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [86]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True

In [87]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )


  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [88]:
max_steps = 3
trained_model_name = f"docs_{max_steps}_steps"
output_dir = trained_model_name

In [89]:
import transformers
training_args = TrainingArguments(

  learning_rate=1.0e-8,
  num_train_epochs=20,
  max_steps=max_steps,
  per_device_train_batch_size=1,
  output_dir=output_dir,
)

In [90]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset = split_dataset['test']

)

max_steps is given, it will override any value given in num_train_epochs


In [91]:
training_output = trainer.train()

Step,Training Loss


In [92]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: docs_3_steps/final


In [93]:
finetuned_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
finetuned_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 1024)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True

In [109]:
test_question = 'What is depression?'

In [110]:
print(inference(test_question, finetuned_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




Depression is a mental disorder that affects the way a person feels. It is a mental disorder that affects the way a person feels. It is a mental disorder that affects the way a person feels. It is a mental disorder that affects the way a person feels. It is a mental disorder that affects the way a person feels. It is a mental disorder that affects the way a person feels. It is a mental disorder that affects the way a person feels. It is
